In [1]:
import random
import math
import os
from pathlib import Path

import pandas as pd
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Image augmentation
from albumentations import Compose, Normalize, HorizontalFlip, VerticalFlip
from albumentations.pytorch import ToTensorV2

from skimage import io
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split

from tqdm import tqdm_notebook


<h3>This way we can call models and model inputs <>.to(device) and have it work regardless if on cpu or gpu</h3>

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
torch.hub.set_dir('/home/ubuntu/prostate')


<h3>Seen a lot of people on Kaggle set all seeds in one place </h3>

In [4]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [5]:
seed_torch(9)

In [39]:
img_size = 128
batch_size = 8
N = 10  # number of tiles per image

# Need to change if putting onto a Kaggle kernel?
TRAIN = '../data/tiles_data/train'
LABELS = '../data/train.csv'

In [8]:
data = pd.read_csv(LABELS).set_index('image_id')

In [9]:
data.shape

(10616, 3)

In [10]:
data.head()

,data_provider,isup_grade,gleason_score
image_id,,,
0005f7aaab2800f6170c399693a96917,karolinska,0,0+0
000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0
0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4
001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4
001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0


<font size="3"> Should check at some point if test set has similar distribution of target labels. Thought I saw in paper that test set was more heavily biased towards Grade 5 images...</>

In [11]:
data.isup_grade.value_counts()

0    2892
1    2666
2    1343
4    1249
3    1242
5    1224
Name: isup_grade, dtype: int64

## Dataset Construction

Only consider images we have processed and stored as tiles in TRAIN folder

In [12]:
image_ids = {filepath[:32] for filepath in os.listdir(TRAIN)}

In [13]:
data = data.loc[image_ids]
data.reset_index(inplace=True)

Lose about 100 images (from IAFoss pre-processing, he only used images with masks)

In [14]:
data.shape

(10516, 4)

In [15]:
subset_idxs = np.random.choice(len(data), 500, replace=False)

In [16]:
subset_data = data.iloc[subset_idxs].reset_index(drop=True)

In [34]:
train, valid = train_test_split(data, test_size=0.3, random_state=9)

In [35]:
train.reset_index(inplace=True, drop=True)
valid.reset_index(inplace=True, drop=True)

In [36]:
train.head()

,image_id,data_provider,isup_grade,gleason_score
0,1d5d638e25521b927dd533db4fadbd46,radboud,2,3+4
1,4932f17e88988f2e6ce80c1fe627f390,karolinska,0,0+0
2,d11c87201830d5e0b853fe783ff28b70,radboud,0,negative
3,7b8d2f39e387bc0a54504377f8318247,radboud,3,4+3
4,5d33e6224a522400d7a4bb76e47fcfbf,karolinska,2,3+4


In [37]:
class TileTrainDataSet(Dataset):
    def __init__(self, df, transform_fn=None):
        self.X = df['image_id']
        self.Y = df['isup_grade']
        self.transform = transform_fn

    def __getitem__(self, idx):
        # Take image id and use the first N tiles (all have the same target label)
        img_id = self.X[idx]
        imgs = []
        for i in range(N):
            img = io.imread(os.path.join(TRAIN,img_id+f"_{i}.png"))
            
            if self.transform:
                augmented = self.transform(image=img)
                img = augmented['image']
            imgs.append(img)
        # Final shape is x:  N x 3 x 128 x 128, y: 1
        x = torch.stack(imgs)
        return x, self.Y[idx]

    def __len__(self):
        return len(self.Y)

In [38]:
def img_transforms(*, partition):
    
    assert partition in ('train', 'valid')
    
    if partition == 'train':
        return Compose([
            HorizontalFlip(p=0.5),  # 50/50 chance of performing horizontal flip
            VerticalFlip(p=0.5),
            # Normalize images according to ResNext specifications
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    
    elif partition == 'valid':
        # Don't flip validation data 
        return Compose([
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

In [40]:
train_ds = TileTrainDataSet(train, transform_fn=img_transforms(partition='train'))

In [41]:
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

In [42]:
valid_ds = TileTrainDataSet(valid, transform_fn=img_transforms(partition='valid'))

In [43]:
valid_dl = DataLoader(valid_ds, batch_size=batch_size, shuffle=True)

## Model

In [44]:
class Model(nn.Module):
    # n=6 represents number of label classes, give better name. 
    # Except for now doing regression instead of classification
    def __init__(self, arch='resnext50_32x4d_ssl', n=6, pre=True):
        super().__init__()
        m = torch.hub.load('facebookresearch/semi-supervised-ImageNet1K-models', arch)
        self.enc = nn.Sequential(*list(m.children())[:-2])  # Remove last two layers from ResNext
        nc = list(m.children())[-1].in_features  # 2048 (last linear layer of resnext50)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.linear1 = nn.Linear(nc,1)
        #self.bn = nn.BatchNorm1d(512)
        #self.dropout = nn.Dropout(0.5)
        #self.linear2 = nn.Linear(512,1)
                                 
    def forward(self, x):
        # Original shape: bs x N x 3 x 128 x 128
        shape = x.shape
        x = x.view(-1,shape[2],shape[3],shape[4])  # bs*N x 3 x 128 x 128
        # C represents output_size from ResNext
        x = self.enc(x)  # bs*N x C x 4 x 4
        
        shape = x.shape
        # concatenate the output for tiles into a single map
        # Need to do in two steps to 1) Separate batch_size and N, 2) Combine N into outer dimensions 
        # Result: bs x C x N*4 x 4
        x = x.view(-1,N,shape[1],shape[2],shape[3]).permute(0,2,1,3,4).contiguous()\
          .view(-1,shape[1],shape[2]*N,shape[3])  
        
        # With 2-D pooling over size 1, reduces last two dimensions to 1 
        x = self.pool(x)  # bs x C x 1 x 1
        # Flatten last three dimensions (result: bs x C)
        x = self.linear1(torch.flatten(x, start_dim=1)) 
        #x = self.bn(x)
        #x = self.dropout(x)
        #x = self.linear2(x)
        # Look at other pre-trained models intended for regression?
        return x


## Train

In [45]:
def save_model(m, p): torch.save(m.state_dict(), p)
    
def load_model(m, p): m.load_state_dict(torch.load(p))

In [46]:
def train_model(model, optimizer, train_dl, epochs,):
    iterations = epochs*len(train_dl)
    pbar = tqdm_notebook(total=iterations)
    best_kappa = 0.0
    
    for i in range(epochs):
        model.train()
        total_loss = 0
        total = 0

        for img, label in train_dl:
            img = img.to(device)
            label = label.to(device).float().unsqueeze(1)
            out = model(img)
            # some suggest since kappa is a quasi-measure of "distance" from true label, 
            # better to calculate MSE regression loss than classification loss
            loss = F.mse_loss(out, label)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            total_loss += label.size(0)*loss.item()
            total += label.size(0)
            pbar.update()
        train_loss = total_loss/total
        val_loss, val_kappa = valid_metrics(model, valid_dl)
        print(f"train_loss {train_loss:.3f} val_loss {val_loss:.3f} val_kappa {val_kappa:.3f}")
        if val_kappa > best_kappa:
            best_kappa = val_kappa
            save_model(model, f"../models/model_{best_kappa:.3f}.pth")
            print(f"New best model")

In [47]:
def valid_metrics(model, valid_dl):
    iterations = len(valid_dl)
    pbar = tqdm_notebook(total=iterations)
    
    model.eval()
    total = 0
    total_loss = 0
    preds = []
    labels = []
    for img, label in valid_dl:
        img = img.to(device)
        batch = label.shape[0]
        out = model(img)
        loss = F.mse_loss(out, label.to(device).float().unsqueeze(1))
        total_loss += batch*(loss.item())
        total += batch
        
        preds.append(out.detach().to('cpu').apply_(threshold).long().numpy())
        labels.append(label.long().numpy())
        pbar.update()
        
    val_loss = total_loss/total
    val_kappa = cohen_kappa_score(np.concatenate(preds), np.concatenate(labels).reshape(-1,1), weights='quadratic')
    return val_loss, val_kappa

In [48]:
def threshold(x):
    return max(
                min(round(x),5)
            ,0)

In [49]:
model = Model()
model.to(device);

Using cache found in /home/ubuntu/prostate/facebookresearch_semi-supervised-ImageNet1K-models_master


In [50]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

In [51]:
train_model(model, optimizer, train_dl, epochs=3)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


train_loss 1.938 val_loss 1.775 val_kappa 0.657
New best model


train_loss 1.706 val_loss 1.421 val_kappa 0.656


train_loss 1.622 val_loss 1.474 val_kappa 0.633


## Step by Step Run Through

In [ ]:
model = Model()

In [ ]:
x, y = next(iter(train_dl))

In [ ]:
y = y.view(-1,1)

In [ ]:
y.shape

In [ ]:
y.view(-1).shape

In [ ]:
x.shape

In [ ]:
shape = x.shape

In [ ]:
x = x.view(-1, shape[2], shape[3], shape[4])

In [ ]:
x.shape

In [ ]:
x = x.float()

In [ ]:
x = model.enc(x)

In [ ]:
x.shape

In [ ]:
shape = x.shape

In [ ]:
# Need to do in two steps so: 1) tiles are separated (320 into 32 & 10), 2) tiles combined into 2nd dimension (10*4)
x = x.view(-1,N,shape[1],shape[2],shape[3]).permute(0,2,1,3,4).contiguous()\
          .view(-1,shape[1],shape[2]*N,shape[3])          

In [ ]:
x.shape

In [ ]:
x = model.pool(x)

In [ ]:
x.shape

In [ ]:
x = torch.flatten(x, start_dim=1)

In [ ]:
x.shape

In [ ]:
x = model.linear1(x)

In [ ]:
x.shape

In [ ]:
x = model.bn(x)

In [ ]:
x.shape

In [ ]:
x = model.dropout(x)

In [ ]:
x.shape

In [ ]:
x = model.linear2(x)

In [ ]:
x.shape